In [ ]:
from graphviz import Digraph
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn
#poly=sklearn.preprocessing.PolynomialFeatures(interaction_only=True,include_bias = False)


In [ ]:
mydata = pd.read_csv('Retail_Train-1553348772483.csv')

In [ ]:
mydata.shape

In [ ]:
mydata.head()

In [ ]:
mydata.dtypes

In [ ]:
mydata.describe()

In [ ]:
corr = mydata.corr()
corr.style.background_gradient(cmap='coolwarm')


In [ ]:
mydata.PersonID.nunique()

In [ ]:
mydata["FamilySize"].value_counts()

In [ ]:
mydata["TransactionMode"].value_counts()

In [ ]:
mydata["Occupation"].value_counts()

In [ ]:
mydata["TransactionMode"].value_counts()

In [ ]:
mydata.describe()

In [ ]:
plt.hist(mydata.Amount,bins=10)
plt.xticks(rotation='vertical')
plt.title('Amount Distribution',pad=10)
plt.xlabel('Amount',labelpad=10)
plt.ylabel('Frequency')


In [ ]:
sns.pairplot(mydata)

In [ ]:
bins=[0,2,5,9]
mydata['Famsize'] = pd.cut(mydata['Famsize'],bins=bins)

In [ ]:
mydata['Famsize'].value_counts()

In [ ]:
mydata['Occupation'] = mydata['Occupation'].astype('category')
mydata['TransactionMode'] = mydata['TransactionMode'].astype('category')
mydata['Area'] = mydata['Area'].astype('category')

In [ ]:
mydata.dtypes

In [ ]:
## Dropping Columns family size and Person ID
mydata = mydata.drop(columns=['PersonID','FamilySize'])

In [ ]:
##Null Values
mydata.isnull().sum()

In [210]:
pd.value_counts(mydata['Quantity'].values)

356      8
316      7
226      7
419      7
321      7
803      7
481      7
86       6
741      6
173      6
1969     6
236      6
568      6
1033     6
106      6
1179     5
241      5
369      5
662      5
511      5
778      5
901      5
454      5
30       5
133      5
218      5
1340     5
1367     5
266      5
1767     5
        ..
2336     1
4385     1
2338     1
293      1
2342     1
2348     1
2352     1
305      1
307      1
10502    1
2308     1
253      1
215      1
2240     1
4289     1
195      1
2248     1
2252     1
207      1
2258     1
2264     1
2296     1
217      1
2266     1
219      1
6378     1
449      1
2292     1
4898     1
4059     1
Length: 1749, dtype: int64

In [214]:
mydata= mydata[~(mydata == 0).any(axis=1)]

In [ ]:
pd.value_counts(mydata['Quantity'].values,sort=False)

In [ ]:
(mydata['DirectVisits']>mydata['OnlineVisits']).value_counts()

In [ ]:
(mydata['DirectVisits']==0).value_counts()

In [ ]:
(mydata['OnlineVisits']==0).value_counts()

In [ ]:
x=mydata['DirectVisits'] - mydata['OnlineVisits']
x
(x==mydata['OnlineVisits']).value_counts()
##Only online only users which is not very significant

In [ ]:
mydata.plot(x='DirectVisits',y='OnlineVisits',style=['o','rx'])

In [ ]:
y= mydata['Amount']
X=mydata[mydata.columns.difference(['Amount'])]

In [ ]:
num_attribute = X.select_dtypes(include='number').columns.tolist()
cat_attributes = X.columns.difference(num_attribute).tolist()

In [ ]:
mydata.dtypes

In [ ]:
num_attribute

In [ ]:
cat_attributes

In [ ]:
print(y.shape)

In [ ]:
print(X.shape)

In [ ]:
X = pd.get_dummies(X)
X.head()

In [ ]:
##Standardization
from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler()
standardizer.fit(X[num_attribute])
X[num_attribute] = standardizer.transform(X[num_attribute])

In [ ]:
from sklearn.model_selection import train_test_split ##X stands for data Y stands for y variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print('X_train',X_train.shape)
print('y_train',y_train.shape)
print('X_test',X_test.shape)

print('y_test',y_test.shape)

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

## Model building
regressor = linear_model.LinearRegression()

regressor.fit(X_train,y_train)

In [ ]:
y_pred = regressor.predict(X_test)
y_pred_train = regressor.predict(X_train)


In [ ]:
def mean_absolute_percentage_error(y_test, y_pred): 
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [ ]:
print('Coefficients: \n', regressor.coef_)
## Train Error
print("Mean absolute percentage error - Train: %.2f"
      % mean_absolute_percentage_error(y_train, y_pred_train))
# The mean squared error
print("Mean absolute percentage error: %.2f"
      % mean_absolute_percentage_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

In [ ]:
import statsmodels.formula.api as sfm

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns

In [ ]:
vif.round(1)

In [ ]:
## model2 building another model after dropping columns which have high colinearity
X_train2 = X_train.drop(columns=['Famsize_(0, 2]','Famsize_(2, 5]'])
X_test2 = X_test.drop(columns=['Famsize_(0, 2]','Famsize_(2, 5]'])
print(X_train2.shape)
print(X_test2.shape)

In [ ]:
X_train2.head()

In [ ]:
regressor2 = linear_model.LinearRegression()
regressor2.fit(X_train2,y_train)

In [ ]:
y_pred2 = regressor2.predict(X_test2)
y_pred_train2 = regressor2.predict(X_train2)

In [ ]:
print('Coefficients: \n', regressor2.coef_)
## Train Error
print("Mean absolute percentage error - Train: %.2f"
      % mean_absolute_percentage_error(y_train, y_pred_train2))
# The mean squared error
print("Mean absolute percentage error: %.2f"
      % mean_absolute_percentage_error(y_test, y_pred2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred2))

In [ ]:
## Trying OLS regression for model3
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
model3 = sm.OLS(y_train,X_train)

In [ ]:
result3 = model3.fit()

In [ ]:
print(result3.summary())

In [ ]:
## building model with ccolumns which have p value <0.5
X_train4 = X_train.drop(columns=['DirectVisits', 'NumberofFrequentItems','OnlineVisits','Area_Area1','Area_Area2','Occupation_1','Occupation_2', 'Occupation_3', 'TransactionMode_1','TransactionMode_2'])
X_test4 = X_test.drop(columns=['DirectVisits', 'NumberofFrequentItems','OnlineVisits','Area_Area1','Area_Area2','Occupation_1','Occupation_2', 'Occupation_3', 'TransactionMode_1','TransactionMode_2'])

print(X_train4.shape)
print(X_test4.shape)

In [ ]:
model4 = sm.OLS(y_train,X_train4)

In [ ]:
result4 = model4.fit()

In [ ]:
print(result4.summary())

In [ ]:
## Model details so far
print("Model1 - Linear Model")
## Train Error
print("Mean absolute percentage error - Train: %.2f"
      % mean_absolute_percentage_error(y_train, y_pred_train))
# The mean squared error
print("Mean absolute percentage error: %.2f"
      % mean_absolute_percentage_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("________________________________________________________________________")


## Model details so far
print("Model2 - Linear Model reduced dimensions after colinearity check")
## Train Error
print("Mean absolute percentage error - Train: %.2f"
      % mean_absolute_percentage_error(y_train, y_pred_train2))
# The mean squared error
print("Mean absolute percentage error: %.2f"
      % mean_absolute_percentage_error(y_test, y_pred2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred2))
print("________________________________________________________________________")



## Model 3
print("Model 3 - OLS Regression")
error3 = math.sqrt(result3.mse_model)
print("Mean squared error: ",error3)
print('R-Square Score: ', result3.rsquared)
print("________________________________________________________________________")

import math
## Model 4

print("Model 4 - OLS Regression after Dimensionality reduction")

error4 = math.sqrt(result4.mse_model)
print("Mean squared error:",error4)
print("R-Square Score:",result4.rsquared)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.cross_validation import train_test_split

In [ ]:
lasso = Lasso()
lasso.fit(X_train,y_train)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)

In [ ]:
print("training score:", train_score)
print ("test score: ", test_score)
print ("number of features used: ", coeff_used)

In [ ]:
lasso001 = Lasso(alpha=0.01, max_iter=10e5)
lasso001.fit(X_train,y_train)

In [ ]:
train_score001=lasso001.score(X_train,y_train)
test_score001=lasso001.score(X_test,y_test)
coeff_used001 = np.sum(lasso001.coef_!=0)
print ("training score for alpha=0.01:", train_score001) 
print ("test score for alpha =0.01: ", test_score001)
print ("number of features used: for alpha =0.01:", coeff_used001)

In [ ]:
lasso00001 = Lasso(alpha=0.0001, max_iter=10e5)
lasso00001.fit(X_train,y_train)

In [ ]:
train_score00001=lasso00001.score(X_train,y_train)
test_score00001=lasso00001.score(X_test,y_test)
coeff_used00001 = np.sum(lasso00001.coef_!=0)
print ("training score for alpha=0.0001:", train_score00001) 
print ("test score for alpha =0.0001: ", test_score00001)
print ("number of features used: for alpha =0.0001:", coeff_used00001)

In [ ]:
plt.subplot(1,2,2)
plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
plt.plot(lasso001.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Lasso; $\alpha = 0.01$') # alpha here is for transparency
plt.plot(lasso00001.coef_,alpha=0.8,linestyle='none',marker='v',markersize=6,color='black',label=r'Lasso; $\alpha = 0.00001$') # alpha here is for transparency
plt.plot(regressor.coef_,alpha=0.7,linestyle='none',marker='o',markersize=5,color='green',label='Linear Regression',zorder=2)
plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=4)
plt.tight_layout()
plt.show()

In [ ]:
### Trying SVM regressor on the data

from sklearn.svm import SVR

## Create an SVC object and print it to see the default arguments
svr = SVR()
svr




In [ ]:


svr_rbf = SVR(kernel= 'rbf', C= 1e3, gamma= 0.1) # defining the support vector regression models
svr_lin = SVR(kernel= 'linear', C= 1e3)
svr_poly = SVR(kernel= 'poly', C= 1e3, degree= 2)
svr_rbf.fit(X_train, y_train) # fitting the data points in the models
svr_lin.fit(X_train, y_train)
svr_poly.fit(X_train, y_train)


In [ ]:
##plt.scatter(X_train, y_train, color= 'black', label= 'Data') # plotting the initial datapoints 
plt.plot(X_train, svr_rbf.predict(X_train), color= 'red', label= 'RBF model') # plotting the line made by the RBF kernel
plt.plot(X_train,svr_lin.predict(X_train), color= 'green', label= 'Linear model') # plotting the line made by linear kernel
plt.plot(X_train,svr_poly.predict(X_train), color= 'blue', label= 'Polynomial model') # plotting the line made by polynomial kernel
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Support Vector Regression')
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
regressortree1 = DecisionTreeRegressor(max_depth=5)
regressortree2 = DecisionTreeRegressor(max_depth=10)

In [ ]:
regressortree1.fit(X_train, y_train)
regressortree2.fit(X_train,y_train)

In [ ]:
y_pred_tree1 = regressortree1.predict(X_test)
y_pred_tree2 = regressortree2.predict(X_test)

In [ ]:
plt.figure()
plt.plot(y_test, y_pred_tree1, color="cornflowerblue",
         label="max_depth=2", linewidth=2)
plt.plot(y_test, y_pred_tree2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

In [ ]:

print("Tree 1")
print("_____________________________")
print('Train score:',regressortree1.score(X_train,y_train))
print('Test score:',regressortree1.score(X_test,y_test))

print("Tree 2")
print("_____________________________")
print('Train score:',regressortree2.score(X_train,y_train))

print('Test Score:', regressortree2.score(X_test,y_test))

In [ ]:
regressortree1.feature_importances_
## Get important Features
fi_dt1 = pd.Series(regressortree1.feature_importances_, index = X_train.columns)
fi_dt1_ordered = feat_importances.nlargest(n=len(fi_dt1))
%matplotlib notebook
feat_importances_ordered.plot(kind='bar')

In [ ]:
regressortree2.feature_importances_
## Get important Features
fi_dt2 = pd.Series(regressortree2.feature_importances_, index = X_train.columns)
fi_dt2_ordered = feat_importances.nlargest(n=len(fi_dt2))
%matplotlib notebook
feat_importances_ordered.plot(kind='bar')

In [ ]:
## Random Forest
from sklearn.ensemble import RandomForestRegressor
# Model initilization
rfc1 = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0, oob_score=True)
# Model training
rfc1.fit(X = X_train,y = y_train)

# Model predictions
rfc1_train_pred = rfc1.predict(X_train)
rfc1_test_pred = rfc1.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Train Score:", rfc1.score)
print("OOB Score", rfc1.oob_score_)

print("Test Score:",rfc1.base_estimator)
##print(accuracy_score(y_test,rfc1_test_pred))
##print(confusion_matrix(y_test,rfc1_test_pred))
##print(classification_report(y_test,rfc1_test_pred))

In [ ]:
rfc1.feature_importances_
## Get important Features
feat_importances = pd.Series(rfc1.feature_importances_, index = X_train.columns)
feat_importances_ordered = feat_importances.nlargest(n=len(feat_importances))
%matplotlib notebook
feat_importances_ordered.plot(kind='bar')

In [ ]:
## Basis this plot building a model based on the feature importance given by the tree
X_train_f = X_train.drop(columns=['Area_Area1','Area_Area2','Occupation_1','Occupation_2', 'Occupation_3', 'TransactionMode_1','TransactionMode_2'])
X_test_f = X_test.drop(columns=['Area_Area1','Area_Area2','Occupation_1','Occupation_2', 'Occupation_3', 'TransactionMode_1','TransactionMode_2'])
reg_f = linear_model.LinearRegression()
reg_f.fit(X_train_f,y_train)

In [ ]:
y_pred_f = reg_f.predict(X_test_f)
y_pred_train_f = reg_f.predict(X_train_f)


In [ ]:
print(X_test_f.shape)
print(X_train_f.shape)


print(y_test.shape)
print(y_pred_f.shape)
print(y_pred_train_f.shape)

In [ ]:
print('Coefficients: \n', reg_f.coef_)
## Train Error
print("Mean absolute percentage error - Train: %.2f"
      % mean_absolute_percentage_error(y_train, y_pred_train_f))
# The mean squared error
print("Mean absolute percentage error - Test: %.2f"
      % mean_absolute_percentage_error(y_test, y_pred_f))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred_f))